In [28]:
#File description & paths to csv's
"""
Code is divided into the following sections:
1. Import data & create dataframes:
    -credit_records_o contains credit_records from csv file, same structure as in the file (long form)
    -application_records_o contains data from application csv, file same structure as in the file
    -credit_records_t is wide form of credit_records_o 
2. Prepare data in application records (i.e. fill in blanks in occupation type, standardize values and more)
3. Enrich credit data with start, end, duration, count of individual statues (0_count, 1_count... 5 ratio) and count all statuses (all_late_count),
ratio of individual status (0_ratio, 1_ratio... 5_ratio) and all late statuse ratio (all_later_ratio)
    - credit_records_t_label
4. Enrich credit data with Active & TOTAL_STATUS & GOOD_BAD CLIENT measures calculated wiht Mateusz method
5. Merge app & labelled credit data and calculate bins for discrete variables (birth date, employment date & amount income)
    - app_and_credit_label_2
6. Functions for GUI in IPWIDGET that take & store user input from ipywidgets
5. 
"""

#read script location and look for folder called data in which there should be files to import
import os
data_dir=os.path.join(os.path.abspath(''),"data")
path_to_credit_records_csv=os.path.join(data_dir,"credit_record.csv")
path_to_application_records_csv=os.path.join(data_dir,"application_record.csv")

In [29]:
### 1. Import of data & dataframe def 
import pandas as pd

#create credit_records_o from data in csv
file=open(path_to_credit_records_csv) 
credit_records_o=pd.read_csv(file) 
file.close()

#create application_records_o from data in csv
file=open(path_to_application_records_csv) 
application_records_o=pd.read_csv(file) 
file.close()

#create wide version of credit_records
credit_records_t=pd.pivot(credit_records_o, index='ID',columns='MONTHS_BALANCE',values='STATUS') 
credit_records_t = credit_records_t[credit_records_t.columns.sort_values(ascending=False)] #reorder months balance column in descending order

In [30]:
### 2. Applcation Data Preparation Asia
application_records_t=application_records_o.copy()
application_records_t.drop("FLAG_MOBIL", axis=1, inplace=True) #dropping as its 1 across all rows
application_records_t["FLAG_OWN_CAR"].replace(["Y","N"],["1","0"],inplace=True) #standardize to same format as other flags i.e. 1 for True and 0 for False
application_records_t["FLAG_OWN_REALTY"].replace(["Y","N"],["1","0"],inplace=True) #standardize to same format as other flags i.e. 1 for True and 0 for False
application_records_t["FLAG_OWN_CAR"]=application_records_t["FLAG_OWN_CAR"].astype(int) #standardize to int format
application_records_t["FLAG_OWN_REALTY"]=application_records_t["FLAG_OWN_REALTY"].astype(int) #standardize to int format
application_records_t["OCCUPATION_TYPE"].fillna(value="Not provided", inplace=True) #fill in blanks in occupation type

In [31]:
### 3. Enrich credit data and define labels 

from collections import Counter
p=credit_records_t.copy() 

#add colulmns with val_count(count of statuses), start (first month of loan), end (last month of loan), duration (loan duration)
p['val_count']=p.apply(lambda x: Counter(x),axis=1) 
grouped_id=credit_records_o.groupby('ID')
p['start_mth']=grouped_id['MONTHS_BALANCE'].min() #month at which the credit started for client
p['end_mth']=grouped_id['MONTHS_BALANCE'].max() #month at which the credit ended for client
p['duration']=abs(p['start_mth']-p['end_mth'])+1 #duration of the credit for client

#add columns with 0,1,2,3,4,5 statuses count and their ratios per client
p['0_count']=p['val_count'].apply(lambda x: x.get('0',0)) 
p['0_ratio']=p['0_count']/p['duration'] 
p['1_count']=p['val_count'].apply(lambda x: x.get('1',0)) 
p['1_ratio']=p['1_count']/p['duration'] 
p['2_count']=p['val_count'].apply(lambda x: x.get('2',0))
p['2_ratio']=p['2_count']/p['duration'] 
p['3_count']=p['val_count'].apply(lambda x: x.get('3',0))
p['3_ratio']=p['3_count']/p['duration'] 
p['4_count']=p['val_count'].apply(lambda x: x.get('4',0))
p['4_ratio']=p['4_count']/p['duration'] 
p['5_count']=p['val_count'].apply(lambda x: x.get('5',0))
p['5_ratio']=p['5_count']/p['duration'] 
# p['all_late_count']=p['val_count'].apply(lambda x: x.get('0',0)+x.get('1',0)+x.get('2',0)+x.get('3',0)+x.get('4',0)+x.get('5',0)) 
p['all_late_count']=p['val_count'].apply(lambda x: x.get('2',0)+x.get('3',0)+x.get('4',0)+x.get('5',0)) 
p['all_late_ratio']=p['all_late_count']/p['duration'] 
p['all_late_flag']=p['all_late_ratio'].apply(lambda x: 1 if x>0.20 else 0)

credit_records_t_label=p.copy()


In [32]:
### 4. Enrich credit data with Mateusz method
df=credit_records_o.copy() 

df["MONTHS_BALANCE"] = df["MONTHS_BALANCE"].abs()

df_last = df.groupby(by="ID")["MONTHS_BALANCE"].min().reset_index(name="LAST_RECORD")
df = pd.merge(df, df_last, on="ID", how="left")

df_all = df.groupby(by="ID")["MONTHS_BALANCE"].count().reset_index(name="NUMBER_OF_RECORDS")
df = pd.merge(df, df_all, on="ID", how="left")

df["MONTHS"] = df["MONTHS_BALANCE"] - df["LAST_RECORD"]

def account_activity(ID, last_record):
    
    """
    Funkcja zwracająca obecny stan konta klienta.
    Konto aktywne - 1
    Konto nieaktywne - 0
    """
        
    if last_record == 0:
        active = 1
    else:
        active = 0
        
        
    return active

df_upgrade = df.copy()
df_upgrade["ACTIVE"] = df_upgrade.apply(lambda df: account_activity(df["ID"], df["LAST_RECORD"]), axis=1)

def month_weight(month, number_of_records):
    
    """
    Funkcja przyporządkowująca wagę danemu miesiącu (month) z uwzględnieniem liczby wszystkich miesięcy (number_of_months).
    Wagi maleją w sposób liniowy, a ich suma daje 1.
    """
    
    k = abs(month) + 1
    n = number_of_records
    
    if n < 4:
        weight = 0
    elif 4 <= n <= 24:
        weight = -2*k/(n*(n+1))+2/n
    else:
        n = 24
        if k <= 24:
            weight = -2*k/(n*(n+1))+2/n
        else:
            weight = 0
              
                
    return weight

def status2number(month, status, number_of_records):
    
    """
    W oparciu o funkcję month_weight, status2number przekształca status z wybranego miesiąca na liczbę z przedziału <-1,1>.

    """
    
    if month <= 23:
        weight = month_weight(month, number_of_records)

        if status is "5":
            value = -1
        elif status is "X":
            value = 0
        elif status is "C":
            value = 1
        else:
            n = int(status)
            value = 1/2**(n+1)-1
    else:
        weight = 1
        value = 0
    
    
    return weight * value

df_upgrade["PARTIAL_STATUS"] = df_upgrade.apply(lambda df: status2number(df["MONTHS"], df["STATUS"], df["NUMBER_OF_RECORDS"]), axis=1)

df_status = df_upgrade.groupby(by = "ID")["PARTIAL_STATUS"].sum().reset_index(name="TOTAL_STATUS")

df_credit = pd.merge(df_upgrade, df_status, on="ID", how="left")

def good_bad_function(ID, status):
    
    """
    Funkcja określająca czy dany klient może otrzymać kredyt.
    """
    
    if status >= 0:
        judgment = 1
    else:
        judgment = 0
    
    
    return judgment

df_credit["GOOD_BAD"] = df_credit.apply(lambda df: good_bad_function(df["ID"], df["TOTAL_STATUS"]), axis=1)

df_result = df_credit[["ID", "ACTIVE", "TOTAL_STATUS", "GOOD_BAD"]].drop_duplicates().copy()

<>:67: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:69: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:71: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:67: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:69: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:71: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\User\AppData\Local\Temp\ipykernel_18528\2813601196.py:67: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if status is "5":
C:\Users\User\AppData\Local\Temp\ipykernel_18528\2813601196.py:69: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif status is "X":
C:\Users\User\AppData\Local\Temp\ipykernel_18528\2813601196.py:71: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif status is "C":


In [33]:
### 5. Merge app & labelled credit data & enrich it with age, empl in year, income, age, emp bins
app_and_credit_label_2=application_records_o.merge(credit_records_t_label,on='ID') #merege of app & credit labelled data
app_and_credit_label_2=app_and_credit_label_2.merge(df_result,on='ID')                 #merege of Mateusz's labells
# app_and_credit_label_2['GOOD_BAD_INV']=app_and_credit_label_2['GOOD_BAD'].apply(lambda x: 1 if x==0 else 0) #this calculater measure if client is bad

app_and_credit_label_2['income_bin']=pd.cut(app_and_credit_label_2['AMT_INCOME_TOTAL'],[50000, 75000,100000,150000,200000,250000,300000,float("inf")])
app_and_credit_label_2['age']=round((app_and_credit_label_2['DAYS_BIRTH']/365),0)
app_and_credit_label_2['age_bin']=pd.cut(app_and_credit_label_2['age'],[-70,-60,-50,-40,-30,-20])
app_and_credit_label_2['length_of_empl']=round((app_and_credit_label_2['DAYS_EMPLOYED']/365),0)
app_and_credit_label_2['length_of_empl_bin']=pd.cut(app_and_credit_label_2['length_of_empl'],[-30,-20,-10,-5,0])



In [34]:
### 6. DASH APP taking features and 2 tagets to compare targets relation

#Rename columns to more friendly format
# app_and_credit_label_2.rename(columns={'CODE_GENDER':'Gender', 'FLAG_OWN_CAR': 'Own Car','FLAG_OWN_REALTY':'Own Realty', 'CNT_CHILDREN':'Children Count', 'AMT_INCOME_TOTAL':'Yearly Income','NAME_INCOME_TYPE':'Income Type', 'NAME_EDUCATION_TYPE':'Education Type','NAME_FAMILY_STATUS':'Family Status', 'NAME_HOUSING_TYPE': 'Housing Type','DAYS_BIRTH':'Days Birth', 'DAYS_EMPLOYED':'Days Employed', 'FLAG_MOBIL':'Mobil', 'FLAG_WORK_PHONE':'Work Phone', 'FLAG_PHONE': 'Phone Flag', 'FLAG_EMAIL':'Email.Flag', 'OCCUPATION_TYPE':'Occupation Type', 'CNT_FAM_MEMBERS':'Family Members #','GOOD_BAD':'bad clients 1', 'all_late_flag':'bad clients 2'},inplace=True)

from dash import dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.express as px

# app = Dash(__name__)
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

l1=app_and_credit_label_2.columns[1:17]
t1=app_and_credit_label_2.columns[-21:-3]
p1=[*range(-60,1,1)]

el1=dbc.Col(dcc.Graph(id='graph1'), md=6)
el2=dbc.Col(dcc.Graph(id='graph2'), md=6)
el3=dbc.Col(dcc.Graph(id='graph3'), md=6)
el4=dbc.Col(dcc.Graph(id='graph4'), md=6)

row0=dbc.Form([dbc.CardHeader('Select Period'), dcc.RangeSlider(-60, 0, 6, value=[-60, 0], id='start-end')])
row1=dbc.Row([el1,el2])
row2=dbc.Row([el3,el4])

SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 100,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

TEXT_STYLE={}

controls = dbc.Form([
        dbc.CardHeader('Select Feature 1'),
        dbc.Card(dcc.Dropdown(l1,l1,id='l1_drop')),
        dbc.CardHeader('Select Feature 2'),
        dbc.Card(dcc.Dropdown(l1,l1,id='l2_drop')),
        dbc.CardHeader('Select target'),
        dbc.Card(dcc.Dropdown(t1,t1,id='l3_drop')),
        dbc.CardHeader('Select target 2'),
        dbc.Card(dcc.Dropdown(t1,t1,id='l4_drop'))
        ])

sidebar = html.Div(
    [
        html.H2('Parameters', style=TEXT_STYLE),
        html.H5('Enter all parameters to see all visualisations'),
        html.Hr(),
        controls
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(
    [
    row0,
    row1,
    row2,
    ],
    style=CONTENT_STYLE
)

# tab1_content=dbc.Card(html.Div([sidebar,content]),className="mt-3")
tab1_content=html.Div([sidebar,content])
tab2_content=dbc.Card(
    dbc.CardBody(
        [
            html.P("This is tab 2!", className="card-text"),
            dbc.Button("Don't click here", color="danger"),
        ]
    ),
    className="mt-3",
)

tabs = dbc.Tabs(
    [
        dbc.Tab(tab1_content, label="Tab 1"),
        dbc.Tab(tab2_content, label="Tab 2"),
        dbc.Tab(
            "This tab's content is never seen", label="Tab 3", disabled=True
        ),
    ]
)

app.layout = tabs

@app.callback(
    Output('graph1', 'figure'),
    Output('graph2', 'figure'),
    Output('graph3', 'figure'),
    Output('graph4', 'figure'),
    # Output('graph5', 'figure'),
    # Output('graph6', 'figure'),
    Input('l1_drop', 'value'),
    Input('l2_drop','value'),
    Input('l3_drop','value'),
    Input('l4_drop','value'),
    [Input('start-end', 'value')]
    )

# def update_figure(l1_val,l2_val,t_val):
#     lst=[]
#     features_cat=['CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','CNT_CHILDREN']
#     # features_cat.pop(features_cat.index(l1_val))
#     for el in features_cat:
#         df=pd.pivot_table(app_and_credit_label_2,index=l1_val,columns=el,values='all_late_ratio',aggfunc='mean').round(4)
#         lst.append(px.bar(df,barmode='group',text_auto=True))
#         # lst.append(px.violin(app_and_credit_label_2,x=l1_val,y='all_late_ratio',box=True))
#     for i in range(len(lst)):
#         lst[i].update_layout(transition_duration=500, hovermode='y unified')
#     return lst

def update_figure(l1_val,l2_val,t_val,t_val1,ss):
    lst=[]
    data=app_and_credit_label_2[(app_and_credit_label_2['start_mth']>=ss[0]) & (app_and_credit_label_2['start_mth']<=ss[1])]
    df=pd.pivot_table(data,index=l1_val,columns=l2_val,values=t_val,aggfunc='mean',margins=True).round(4)
    lst.append(px.bar(df,barmode='group',text_auto=True,title="{} bad clients % by {} and {}".format(t_val,l1_val,l2_val))),
    lst.append(px.imshow(df,color_continuous_scale='RdBu_r',text_auto=".2%",title="{} bad clients % by {} and {}".format(t_val,l1_val,l2_val))),
    df=pd.pivot_table(data,index=l1_val,columns=l2_val,values=t_val1,aggfunc='mean',margins=True).round(4)
    lst.append(px.bar(df,barmode='group',text_auto=True,title="{} bad clients % by {} and {}".format(t_val1,l1_val,l2_val))),
    lst.append(px.imshow(df,color_continuous_scale='RdBu_r',text_auto=True,title="{} bad clients % by {} and {}".format(t_val1,l1_val,l2_val)))

#     # lst.append(px.histogram(df[l1_val]))
#     # lst.append(px.histogram(df[l2_val]))
#     # lst.append(px.violin(app_and_credit_label_2,x=l1_val,y='all_late_ratio',box=True))
    for i in range(len(lst)):
        lst[i].update_layout(transition_duration=500)
        lst[i].layout.yaxis.tickformat = ',.2%' #show things as percentage with 2 decimal places
    lst[2].update_xaxes(type='category') #heatmap charts weren't working correclty withotu making axes categorical
    return lst

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Mar/2022 20:12:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:12:23] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:12:23] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:12:23] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:12:23] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:12:23] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 20:12:23] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\dash\dash.py", line 1344, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\dash\_callback.py", line 151, in add_context
   

127.0.0.1 - - [13/Mar/2022 20:12:23] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\dash\dash.py", line 1344, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\dash\_callback.py", line 151, in add_context
   

127.0.0.1 - - [13/Mar/2022 20:12:25] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\dash\dash.py", line 1344, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\dash\_callback.py", line 151, in add_context
   

127.0.0.1 - - [13/Mar/2022 20:12:27] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\flask\app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\dash\dash.py", line 1344, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\User\anaconda3\envs\kivy_venv\lib\site-packages\dash\_callback.py", line 151, in add_context
   

127.0.0.1 - - [13/Mar/2022 20:12:32] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [13/Mar/2022 20:12:35] "POST /_dash-update-component HTTP/1.1" 200 -
